In [1]:
import json

import pandas as pd
import numpy as np

import redis
from redis.commands.search.field import NumericField, TextField, TagField, GeoField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query, NumericFilter

In [2]:
REDIS_HOST = '127.0.0.1'
REDIS_PORT = 6379
REDIS_PASSWORD = ''

conn = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, encoding='utf-8', decode_responses=True)

# JSON Data

In [3]:
res = conn.json().set("anuncio:1", "$", {'titulo': 'Preciosa casa en venta en Lomas de Chapultepec', 
                                           'tipo': 'casa', 'operacion': 'venta', 
                                           'precio': {'venta': 50000000}, 
                                           'amenidades': ['Vigilancia', 'Jardín', 'Alberca'], 
                                           'coordenadas': '-99.21, 19.42', 
                                           'direccion': 'Lomas de Chapultepec, Miguel Hidalgo, Ciudad de México'})

res = conn.json().set("anuncio:2", "$", {'titulo': 'Departamento en renta en Roma Norte a tres calles del Metro Sevilla', 
                                           'tipo': 'departamento', 'operacion': 'sevilla', 
                                           'precio': {'renta': 15000},
                                           'amenidades': ['Vigilancia', 'Parrilla'], 
                                           'coordenadas': '-99.16, 19.42',
                                           'direccion': 'Roma Norte, Cuauhtémoc, Ciudad de México'})

res = conn.json().set("anuncio:3", "$", {'titulo': 'Departanto en renta con vista al parque', 
                                           'tipo': 'departamento', 'operacion': 'renta', 
                                           'precio': {'renta': 18000}, 
                                           'amenidades': ['Vigilancia', 'Terraza', 'Roof Garden'], 
                                           'coordenadas': '-99.14, 19.43',
                                           'direccion': 'Juarez, Cuauhtémoc, Ciudad de México'})

res = conn.json().set("anuncio:4", "$", {'titulo': 'Casa remodelada en venta en Lomas de Chapultepec', 
                                           'tipo': 'casa', 'operacion': 'venta', 
                                           'precio': {'venta': 60000000}, 
                                           'amenidades': ['Jardín', 'Parrilla'], 
                                           'coordenadas': '-99.20, 19.42',
                                           'direccion': 'Lomas de Chapultepec, Miguel Hidalgo, Ciudad de México'})

res = conn.json().set("anuncio:5", "$", {'titulo': 'Casa en renta cerca de escuelas en Coyoacán', 
                                           'tipo': 'casa', 'operacion': 'renta', 
                                           'precio': {'renta': 30000}, 
                                           'amenidades': ['Jardín'], 
                                           'coordenadas': '-99.16, 19.36',
                                           'direccion': 'Santa Cruz Atoyac, Coyoacán, Ciudad de México'})

res = conn.json().set("anuncio:6", "$", {'titulo': 'Departamento en renta cerca del centro de Coyoacán', 
                                           'tipo': 'departamento', 'operacion': 'renta', 
                                           'precio': {'renta': 15000}, 
                                           'amenidades': ['Vigilancia', 'Roof Garden', 'Parrilla'], 
                                           'coordenadas': '-99.16, 19.36',
                                           'direccion': 'Santa Cruz Atoyac, Coyoacán, Ciudad de México'})

res = conn.json().set("anuncio:7", "$", {'titulo': 'Departamento en venta cerca en Tlatelolco a unos pasos del metro', 
                                           'tipo': 'departamento', 'operacion': 'venta', 
                                           'precio': {'venta': 1500000}, 
                                           'amenidades': None, 
                                           'coordenadas': '-99.14, 19.45',
                                           'direccion': 'Tlatelolco, Cuauhtémoc, Ciudad de México'})

res = conn.json().set("anuncio:8", "$", {'titulo': 'Oportunidad de compra de nuevo departamento en exlusiva zona', 
                                           'tipo': 'departamento', 'operacion': 'venta', 
                                           'precio': {'venta': 7000000}, 
                                           'amenidades': ['Vigilancia', 'Acabados de Lujo', 'Terraza'], 
                                           'coordenadas': '-99.19, 19.43',
                                           'direccion': 'Polanco, Miguel Hidalgo, Ciudad de México'})

res = conn.json().set("anuncio:9", "$", {'titulo': 'Departamento ideal para estudiantes cerca de UNAM', 
                                           'tipo': 'departamento', 'operacion': 'renta', 
                                           'precio': {'renta': 7000}, 
                                           'amenidades': None, 
                                           'coordenadas': '-99.17, 19.34',
                                           'direccion': 'Santa Cararina, Coyoacán, Ciudad de México'})

res = conn.json().set("anuncio:10", "$", {'titulo': 'Casa Antigua en venta, excelente oportunidad', 
                                           'tipo': 'casa', 'operacion': 'venta', 
                                           'precio': {'venta': 40000000}, 
                                           'amenidades': ['Parrilla', 'Jardín'], 
                                           'coordenadas': '-99.14, 19.35',
                                           'direccion': 'Churubusco, Coyoacán, Ciudad de México'})

print(res)

True


# Esquema e Indice

In [4]:
#INDEX GENERAL
try:
    conn.ft("idx:anuncio").dropindex()

except: 
    pass

schema = (
    TextField("$.titulo", as_name="titulo"),
    TagField("$.tipo", as_name="tipo"),
    TagField("$.operacion", as_name="operacion"),
    NumericField("$.precio.venta", as_name="precio_venta"),
    NumericField("$.precio.renta", as_name="precio_renta"),
    GeoField("$.coordenadas", as_name="coordenadas"),
    TextField("$.direccion", as_name="direccion"),
    TagField("$.amenidades.*", as_name='amenidades')
)

index = conn.ft("idx:anuncio")
index.create_index(schema, definition=IndexDefinition(prefix=["anuncio:"], index_type=IndexType.JSON))

'OK'

# Busqueda por texto

In [5]:
#WILDCARD
results = index.search(Query("*"))
print("Documents found:", results.total)

Documents found: 10


In [6]:
#PREFIJO
results = index.search(Query('@direccion:Chapultepec*'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 2 

{"titulo":"Preciosa casa en venta en Lomas de Chapultepec","tipo":"casa","operacion":"venta","precio":{"venta":50000000},"amenidades":["Vigilancia","Jardín","Alberca"],"coordenadas":"-99.21, 19.42","direccion":"Lomas de Chapultepec, Miguel Hidalgo, Ciudad de México"}
_____________________

{"titulo":"Casa remodelada en venta en Lomas de Chapultepec","tipo":"casa","operacion":"venta","precio":{"venta":60000000},"amenidades":["Jardín","Parrilla"],"coordenadas":"-99.20, 19.42","direccion":"Lomas de Chapultepec, Miguel Hidalgo, Ciudad de México"}
_____________________



In [7]:
#TEXTO PARCIAL
results = index.search(Query('@direccion:Coyoacán'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 4 

{"titulo":"Departamento en renta cerca del centro de Coyoacán","tipo":"departamento","operacion":"renta","precio":{"renta":15000},"amenidades":["Vigilancia","Roof Garden","Parrilla"],"coordenadas":"-99.16, 19.36","direccion":"Santa Cruz Atoyac, Coyoacán, Ciudad de México"}
_____________________

{"titulo":"Casa Antigua en venta, excelente oportunidad","tipo":"casa","operacion":"venta","precio":{"venta":40000000},"amenidades":["Parrilla","Jardín"],"coordenadas":"-99.14, 19.35","direccion":"Churubusco, Coyoacán, Ciudad de México"}
_____________________

{"titulo":"Casa en renta cerca de escuelas en Coyoacán","tipo":"casa","operacion":"renta","precio":{"renta":30000},"amenidades":["Jardín"],"coordenadas":"-99.16, 19.36","direccion":"Santa Cruz Atoyac, Coyoacán, Ciudad de México"}
_____________________

{"titulo":"Departamento ideal para estudiantes cerca de UNAM","tipo":"departamento","operacion":"renta","precio":{"renta":7000},"amenidades":null,"coordenadas":"-99.17,

In [8]:
#TEXTO TOTAL
results = index.search(Query('@titulo:"Casa en renta cerca de escuelas en Coyoacán"'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 1 

{"titulo":"Casa en renta cerca de escuelas en Coyoacán","tipo":"casa","operacion":"renta","precio":{"renta":30000},"amenidades":["Jardín"],"coordenadas":"-99.16, 19.36","direccion":"Santa Cruz Atoyac, Coyoacán, Ciudad de México"}
_____________________



In [9]:
#MULTIPLES TEXTOS
results = index.search(Query('@titulo:oportunidad|estudiante'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 3 

{"titulo":"Casa Antigua en venta, excelente oportunidad","tipo":"casa","operacion":"venta","precio":{"venta":40000000},"amenidades":["Parrilla","Jardín"],"coordenadas":"-99.14, 19.35","direccion":"Churubusco, Coyoacán, Ciudad de México"}
_____________________

{"titulo":"Departamento ideal para estudiantes cerca de UNAM","tipo":"departamento","operacion":"renta","precio":{"renta":7000},"amenidades":null,"coordenadas":"-99.17, 19.34","direccion":"Santa Cararina, Coyoacán, Ciudad de México"}
_____________________

{"titulo":"Oportunidad de compra de nuevo departamento en exlusiva zona","tipo":"departamento","operacion":"venta","precio":{"venta":7000000},"amenidades":["Vigilancia","Acabados de Lujo","Terraza"],"coordenadas":"-99.19, 19.43","direccion":"Polanco, Miguel Hidalgo, Ciudad de México"}
_____________________



In [10]:
#FUZZY
results = index.search(Query('@direccion:%Polinco%'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 1 

{"titulo":"Oportunidad de compra de nuevo departamento en exlusiva zona","tipo":"departamento","operacion":"venta","precio":{"venta":7000000},"amenidades":["Vigilancia","Acabados de Lujo","Terraza"],"coordenadas":"-99.19, 19.43","direccion":"Polanco, Miguel Hidalgo, Ciudad de México"}
_____________________



# Busqueda por tags y espacial

In [11]:
#TAGS
results = index.search(Query('@amenidades:{Parrilla|Roof Garden}').sort_by('amenidades'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 5 

{"titulo":"Casa remodelada en venta en Lomas de Chapultepec","tipo":"casa","operacion":"venta","precio":{"venta":60000000},"amenidades":["Jardín","Parrilla"],"coordenadas":"-99.20, 19.42","direccion":"Lomas de Chapultepec, Miguel Hidalgo, Ciudad de México"}
_____________________

{"titulo":"Casa Antigua en venta, excelente oportunidad","tipo":"casa","operacion":"venta","precio":{"venta":40000000},"amenidades":["Parrilla","Jardín"],"coordenadas":"-99.14, 19.35","direccion":"Churubusco, Coyoacán, Ciudad de México"}
_____________________

{"titulo":"Departamento en renta en Roma Norte a tres calles del Metro Sevilla","tipo":"departamento","operacion":"sevilla","precio":{"renta":15000},"amenidades":["Vigilancia","Parrilla"],"coordenadas":"-99.16, 19.42","direccion":"Roma Norte, Cuauhtémoc, Ciudad de México"}
_____________________

{"titulo":"Departanto en renta con vista al parque","tipo":"departamento","operacion":"renta","precio":{"renta":18000},"amenidades":["Vigila

In [12]:
#GEOESPACIAL
results = index.search(Query('@coordenadas:[-99.20 19.47 7 km]'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 5 

{"titulo":"Departamento en venta cerca en Tlatelolco a unos pasos del metro","tipo":"departamento","operacion":"venta","precio":{"venta":1500000},"amenidades":null,"coordenadas":"-99.14, 19.45","direccion":"Tlatelolco, Cuauhtémoc, Ciudad de México"}
_____________________

{"titulo":"Departamento en renta en Roma Norte a tres calles del Metro Sevilla","tipo":"departamento","operacion":"sevilla","precio":{"renta":15000},"amenidades":["Vigilancia","Parrilla"],"coordenadas":"-99.16, 19.42","direccion":"Roma Norte, Cuauhtémoc, Ciudad de México"}
_____________________

{"titulo":"Preciosa casa en venta en Lomas de Chapultepec","tipo":"casa","operacion":"venta","precio":{"venta":50000000},"amenidades":["Vigilancia","Jardín","Alberca"],"coordenadas":"-99.21, 19.42","direccion":"Lomas de Chapultepec, Miguel Hidalgo, Ciudad de México"}
_____________________

{"titulo":"Casa remodelada en venta en Lomas de Chapultepec","tipo":"casa","operacion":"venta","precio":{"venta":60000

# Filtros numericos

In [14]:
#NUMERICO
results = index.search(Query('@operacion:{venta}').add_filter(NumericFilter('precio_venta', 0, 10000000)).sort_by('precio_venta'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 2 

{"titulo":"Departamento en venta cerca en Tlatelolco a unos pasos del metro","tipo":"departamento","operacion":"venta","precio":{"venta":1500000},"amenidades":null,"coordenadas":"-99.14, 19.45","direccion":"Tlatelolco, Cuauhtémoc, Ciudad de México"}
_____________________

{"titulo":"Oportunidad de compra de nuevo departamento en exlusiva zona","tipo":"departamento","operacion":"venta","precio":{"venta":7000000},"amenidades":["Vigilancia","Acabados de Lujo","Terraza"],"coordenadas":"-99.19, 19.43","direccion":"Polanco, Miguel Hidalgo, Ciudad de México"}
_____________________



In [15]:
#NUMERICO EXACTO
results = index.search(Query('@precio_venta:[40000000 40000000]'))
print("Documents found:", results.total, '\n')
for result in results.docs:
    print(result.json)
    print('_____________________\n')

Documents found: 1 

{"titulo":"Casa Antigua en venta, excelente oportunidad","tipo":"casa","operacion":"venta","precio":{"venta":40000000},"amenidades":["Parrilla","Jardín"],"coordenadas":"-99.14, 19.35","direccion":"Churubusco, Coyoacán, Ciudad de México"}
_____________________

